# 3 Pointers Made against scoring.csv

### Import packages

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
from scipy.stats import pearsonr
import itertools

pd.set_option("display.max_columns", None)

### Set working directory

In [2]:
# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

# Change working directory
os.chdir('/Users/tyler/OneDrive/Documents/Python/NBA')

# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA\backend\analysis
Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA


## Exploratory Data Analysis

### Import data

In [23]:
df = pd.read_csv('backend/data/scoring.csv').drop(['Unnamed: 0'], axis=1)

### Basic exploration

In [25]:
df.loc[:, 'target'] = df.loc[:, 'q3']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38288 entries, 0 to 38287
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     38288 non-null  object 
 1   visitor  38288 non-null  object 
 2   home     38288 non-null  object 
 3   team     38288 non-null  int64  
 4   q1       38288 non-null  int64  
 5   q2       38288 non-null  int64  
 6   q3       38288 non-null  int64  
 7   q4       38288 non-null  int64  
 8   ot1      2288 non-null   float64
 9   ot2      338 non-null    float64
 10  ot3      62 non-null     float64
 11  ot4      10 non-null     float64
 12  final    38288 non-null  int64  
 13  target   38288 non-null  int64  
dtypes: float64(4), int64(7), object(3)
memory usage: 4.1+ MB


In [26]:
df = df.dropna(axis=1)

In [27]:
df.sample(5)

,date,visitor,home,team,q1,q2,q3,q4,final,target
19775,"Sun, Mar 9, 2014",Portland Trail Blazers,Houston Rockets,0,30,24,31,21,113,31
34449,"Mon, Dec 16, 2019",Cleveland Cavaliers,Toronto Raptors,0,29,25,30,29,113,30
37384,"Fri, Apr 2, 2021",Oklahoma City Thunder,Phoenix Suns,1,43,30,30,37,140,30
26288,"Sun, Nov 27, 2016",Los Angeles Clippers,Indiana Pacers,1,23,27,26,15,91,26
29134,"Wed, Dec 6, 2017",Minnesota Timberwolves,Los Angeles Clippers,1,22,27,31,27,107,31


In [28]:
# Convert 'date' column to Date object
df['date'] = pd.to_datetime(df['date'])

# Conver 'team' column to Team Name
df['team'] = np.where(df['team'], df['home'], df['visitor'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38288 entries, 0 to 38287
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     38288 non-null  datetime64[ns]
 1   visitor  38288 non-null  object        
 2   home     38288 non-null  object        
 3   team     38288 non-null  object        
 4   q1       38288 non-null  int64         
 5   q2       38288 non-null  int64         
 6   q3       38288 non-null  int64         
 7   q4       38288 non-null  int64         
 8   final    38288 non-null  int64         
 9   target   38288 non-null  int64         
dtypes: datetime64[ns](1), int64(6), object(3)
memory usage: 2.9+ MB


### First half data

In [29]:
df['h1'] = df['q1'] + df['q2']
df.sample(5)

,date,visitor,home,team,q1,q2,q3,q4,final,target,h1
11952,2011-02-02,Milwaukee Bucks,Phoenix Suns,Phoenix Suns,25,27,19,21,92,19,52
11953,2011-02-02,Milwaukee Bucks,Phoenix Suns,Milwaukee Bucks,17,14,22,24,77,22,31
10108,2010-03-31,Sacramento Kings,Minnesota Timberwolves,Minnesota Timberwolves,31,22,34,21,108,34,53
17829,2013-05-06,Golden State Warriors,San Antonio Spurs,Golden State Warriors,28,25,39,14,127,39,53
8216,2009-11-18,Detroit Pistons,Portland Trail Blazers,Portland Trail Blazers,21,27,22,17,87,22,48


# Dataframe of team's last 15 performances

In [30]:
# Return ten lastest dates team played
def last_15_date(team, date):
    schedule = df[df['team'] == team].sort_values(by='date').reset_index()
    date_index = schedule[schedule['date'] == date].index[0]
    if date_index - 15 < 0:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
    else:
        date_1, date_2 = schedule.iloc[date_index - 1]['date'], schedule.iloc[date_index - 2]['date']
        date_3, date_4 = schedule.iloc[date_index - 3]['date'], schedule.iloc[date_index - 4]['date']
        date_5, date_6 = schedule.iloc[date_index - 5]['date'], schedule.iloc[date_index - 6]['date']
        date_7, date_8 = schedule.iloc[date_index - 7]['date'], schedule.iloc[date_index - 8]['date']
        date_9, date_10 = schedule.iloc[date_index - 9]['date'], schedule.iloc[date_index - 10]['date']
        date_11, date_12 = schedule.iloc[date_index - 11]['date'], schedule.iloc[date_index - 12]['date']
        date_13, date_14 = schedule.iloc[date_index - 13]['date'], schedule.iloc[date_index - 14]['date']
        date_15 = schedule.iloc[date_index - 15]['date']
        return date_1, date_2, date_3, date_4, date_5, date_6, date_7, date_8, date_9, date_10, date_11, date_12, date_13, date_14, date_15

df['dates'] = df.apply(lambda x: last_15_date(x.team, x.date), axis=1)
df['date_1'], df['date_2'] = df['dates'].apply(lambda x: x[0]), df['dates'].apply(lambda x: x[1])
df['date_3'], df['date_4'] = df['dates'].apply(lambda x: x[2]), df['dates'].apply(lambda x: x[3])
df['date_5'], df['date_6'] = df['dates'].apply(lambda x: x[4]), df['dates'].apply(lambda x: x[5])
df['date_7'], df['date_8'] = df['dates'].apply(lambda x: x[6]), df['dates'].apply(lambda x: x[7])
df['date_9'], df['date_10'] = df['dates'].apply(lambda x: x[8]), df['dates'].apply(lambda x: x[9])
df['date_11'], df['date_12'] = df['dates'].apply(lambda x: x[10]), df['dates'].apply(lambda x: x[11])
df['date_13'], df['date_14'] = df['dates'].apply(lambda x: x[12]), df['dates'].apply(lambda x: x[13])
df['date_15'] = df['dates'].apply(lambda x: x[14])

In [36]:
# X and y column names to merge on
x_cols = ['date', 'team', 'q1', 'q2', 'h1', 'q3', 'q4', 'final']

last_15_games = df.copy()
X = df[x_cols].copy()

# Dataframe of target (3pt made by each team) and of variables (last 5 games stats for each team)
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

## Feature Engineering

In [37]:
# Create season variable
def season(month, year):
    if (month in list(range(10, 13)) and year == 2006) or (month in list(range(1, 7)) and year == 2007):
        return 2006
    elif (month in list(range(10, 13)) and year == 2007) or (month in list(range(1, 7)) and year == 2008):
        return 2007
    elif (month in list(range(10, 13)) and year == 2008) or (month in list(range(1, 7)) and year == 2009):
        return 2008
    elif (month in list(range(10, 13)) and year == 2009) or (month in list(range(1, 7)) and year == 2010):
        return 2009
    elif (month in list(range(10, 13)) and year == 2010) or (month in list(range(1, 7)) and year == 2011):
        return 2010
    elif (month in list(range(10, 11)) and year == 2011) or (month in list(range(1, 7)) and year == 2012):
        return 2011
    elif (month in list(range(10, 13)) and year == 2012) or (month in list(range(1, 7)) and year == 2013):
        return 2012
    elif (month in list(range(10, 13)) and year == 2013) or (month in list(range(1, 7)) and year == 2014):
        return 2013
    elif (month in list(range(10, 13)) and year == 2014) or (month in list(range(1, 7)) and year == 2015):
        return 2014
    elif (month in list(range(10, 13)) and year == 2015) or (month in list(range(1, 7)) and year == 2016):
        return 2015
    elif (month in list(range(10, 13)) and year == 2016) or (month in list(range(1, 7)) and year == 2017):
        return 2016
    elif (month in list(range(10, 13)) and year == 2017) or (month in list(range(1, 7)) and year == 2018):
        return 2017
    elif (month in list(range(10, 13)) and year == 2018) or (month in list(range(1, 7)) and year == 2019):
        return 2018
    elif (month in list(range(10, 13)) and year == 2019) or (month in list(range(1, 11)) and year == 2020):
        return 2019
    elif (month in list(range(12, 13)) and year == 2020) or (month in list(range(1, 8)) and year == 2021):
        return 2020

df['season'] = df['date'].apply(lambda x: season(x.month, x.year))
last_15_games['season'] = last_15_games['date'].apply(lambda x: season(x.month, x.year))
last_15_games.sample()

,date,visitor,home,team,q1,q2,q3,q4,final,target,h1,dates,date_1,date_2,date_3,date_4,date_5,date_6,date_7,date_8,date_9,date_10,date_11,date_12,date_13,date_14,date_15,season,date_1,q1_1,q2_1,h1_1,q3_1,q4_1,final_1,date_2,q1_2,q2_2,h1_2,q3_2,q4_2,final_2,date_3,q1_3,q2_3,h1_3,q3_3,q4_3,final_3,date_4,q1_4,q2_4,h1_4,q3_4,q4_4,final_4,date_5,q1_5,q2_5,h1_5,q3_5,q4_5,final_5,date_6,q1_6,q2_6,h1_6,q3_6,q4_6,final_6,date_7,q1_7,q2_7,h1_7,q3_7,q4_7,final_7,date_8,q1_8,q2_8,h1_8,q3_8,q4_8,final_8,date_9,q1_9,q2_9,h1_9,q3_9,q4_9,final_9,date_10,q1_10,q2_10,h1_10,q3_10,q4_10,final_10,date_11,q1_11,q2_11,h1_11,q3_11,q4_11,final_11,date_12,q1_12,q2_12,h1_12,q3_12,q4_12,final_12,date_13,q1_13,q2_13,h1_13,q3_13,q4_13,final_13,date_14,q1_14,q2_14,h1_14,q3_14,q4_14,final_14,date_15,q1_15,q2_15,h1_15,q3_15,q4_15,final_15
37546,2021-04-12,Denver Nuggets,Golden State Warriors,Golden State Warriors,33,23,43,17,116,43,56,"(2021-04-10 00:00:00, 2021-04-09 00:00:00, 202...",2021-04-10,2021-04-09,2021-04-06,2021-04-04,2021-04-02,2021-04-01,2021-03-29,2021-03-26,2021-03-25,2021-03-23,2021-03-20,2021-03-19,2021-03-17,2021-03-15,2021-03-14,2020.0,2021-04-10,32.0,33.0,65.0,29.0,31.0,125.0,2021-04-09,23.0,27.0,50.0,29.0,28.0,107.0,2021-04-06,28.0,25.0,53.0,30.0,39.0,122.0,2021-04-04,24.0,35.0,59.0,23.0,29.0,111.0,2021-04-02,26.0,16.0,42.0,14.0,21.0,77.0,2021-04-01,24.0,30.0,54.0,27.0,28.0,109.0,2021-03-29,34.0,31.0,65.0,31.0,20.0,116.0,2021-03-26,26.0,26.0,52.0,32.0,24.0,108.0,2021-03-25,37.0,24.0,61.0,25.0,33.0,119.0,2021-03-23,17.0,28.0,45.0,40.0,13.0,98.0,2021-03-20,24.0,27.0,51.0,28.0,24.0,103.0,2021-03-19,34.0,33.0,67.0,24.0,25.0,116.0,2021-03-17,31.0,37.0,68.0,18.0,22.0,108.0,2021-03-15,26.0,24.0,50.0,23.0,24.0,97.0,2021-03-14,33.0,34.0,67.0,28.0,36.0,131.0


In [38]:
# Calculate z-score
def z_score(value, mean, std):
    return (value - mean) / std

In [39]:
# Season average for each stat
season_avgs = df.groupby(['team', 'season']).aggregate(['mean', 'std']).reset_index()
season_avgs.columns = ["_".join([tup[0], tup[1]]) if len(tup[1]) > 0 else tup[0] for tup in season_avgs.columns]
last_15_games = pd.merge(last_15_games, season_avgs, left_on=['team', 'season'], right_on=['team', 'season'])
last_15_games.sample()

,date,visitor,home,team,q1,q2,q3,q4,final,target,h1,dates,date_1,date_2,date_3,date_4,date_5,date_6,date_7,date_8,date_9,date_10,date_11,date_12,date_13,date_14,date_15,season,date_1,q1_1,q2_1,h1_1,q3_1,q4_1,final_1,date_2,q1_2,q2_2,h1_2,q3_2,q4_2,final_2,date_3,q1_3,q2_3,h1_3,q3_3,q4_3,final_3,date_4,q1_4,q2_4,h1_4,q3_4,q4_4,final_4,date_5,q1_5,q2_5,h1_5,q3_5,q4_5,final_5,date_6,q1_6,q2_6,h1_6,q3_6,q4_6,final_6,date_7,q1_7,q2_7,h1_7,q3_7,q4_7,final_7,date_8,q1_8,q2_8,h1_8,q3_8,q4_8,final_8,date_9,q1_9,q2_9,h1_9,q3_9,q4_9,final_9,date_10,q1_10,q2_10,h1_10,q3_10,q4_10,final_10,date_11,q1_11,q2_11,h1_11,q3_11,q4_11,final_11,date_12,q1_12,q2_12,h1_12,q3_12,q4_12,final_12,date_13,q1_13,q2_13,h1_13,q3_13,q4_13,final_13,date_14,q1_14,q2_14,h1_14,q3_14,q4_14,final_14,date_15,q1_15,q2_15,h1_15,q3_15,q4_15,final_15,q1_mean,q1_std,q2_mean,q2_std,q3_mean,q3_std,q4_mean,q4_std,final_mean,final_std,target_mean,target_std,h1_mean,h1_std
20413,2014-04-02,Golden State Warriors,San Antonio Spurs,Golden State Warriors,18,24,25,23,90,25,42,"(2014-04-01 00:00:00, 2014-03-30 00:00:00, 201...",2014-04-01,2014-03-30,2014-03-28,2014-03-22,2014-03-20,2014-03-18,2014-03-16,2014-03-14,2014-03-12,2014-03-11,2014-03-09,2014-03-07,2014-03-05,2014-03-04,2014-03-02,2013.0,2014-04-01,30.0,23.0,53.0,32.0,23.0,122.0,2014-03-30,32.0,12.0,44.0,22.0,18.0,84.0,2014-03-28,30.0,23.0,53.0,26.0,21.0,100.0,2014-03-22,22.0,25.0,47.0,22.0,21.0,90.0,2014-03-20,29.0,24.0,53.0,29.0,33.0,115.0,2014-03-18,30.0,27.0,57.0,30.0,16.0,103.0,2014-03-16,23.0,21.0,44.0,33.0,36.0,113.0,2014-03-14,32.0,21.0,53.0,18.0,23.0,94.0,2014-03-12,29.0,27.0,56.0,23.0,19.0,98.0,2014-03-11,29.0,26.0,55.0,24.0,29.0,108.0,2014-03-09,22.0,34.0,56.0,38.0,19.0,113.0,2014-03-07,32.0,34.0,66.0,19.0,26.0,111.0,2014-03-05,30.0,31.0,61.0,23.0,24.0,108.0,2014-03-04,25.0,23.0,48.0,25.0,25.0,98.0,2014-03-02,28.0,22.0,50.0,28.0,20.0,98.0,26.550562,5.458545,26.505618,5.170114,25.573034,5.762117,25.101124,6.039726,104.426966,11.801278,25.573034,5.762117,53.05618,7.917701


In [40]:
stats = ['q1', 'q2', 'h1', 'q3', 'q4', 'final']

for stat in stats:
    # 3 day trend
    last_15_games[stat + '_trend_3'] = \
        (z_score(last_15_games[stat + '_1'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_2'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_3'], last_15_games[stat + '_mean'], last_15_games[stat + '_std'])) / 3
    
    # 5 day trend
    last_15_games[stat + '_trend_5'] = \
        (z_score(last_15_games[stat + '_1'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_2'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_3'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_4'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_5'], last_15_games[stat + '_mean'], last_15_games[stat + '_std'])) / 5
    
    # 10 day trend
    last_15_games[stat + '_trend_10'] = \
        (z_score(last_15_games[stat + '_1'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_2'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_3'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_4'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_5'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_6'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_7'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_8'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_9'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_10'], last_15_games[stat + '_mean'], last_15_games[stat + '_std'])) / 10

### Last Performance

In [41]:
stats = ['q1', 'q2', 'h1', 'q3', 'q4', 'final']
dates = ['_1']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_game = last_15_games[cols].copy()

for stat in stats:
    last_game[stat] = 0
    for date in dates:
        last_game[stat] = last_game[stat] + last_game[stat + date]
    
    last_game[stat] = last_game[stat] / len(dates)

last_game = last_game.dropna(axis=0)
last_game.sample(5)

,date,visitor,home,team,target,q1_1,q2_1,h1_1,q3_1,q4_1,final_1,q1_trend_3,q1_trend_5,q1_trend_10,q2_trend_3,q2_trend_5,q2_trend_10,h1_trend_3,h1_trend_5,h1_trend_10,q3_trend_3,q3_trend_5,q3_trend_10,q4_trend_3,q4_trend_5,q4_trend_10,final_trend_3,final_trend_5,final_trend_10,q1,q2,h1,q3,q4,final
17782,2013-04-17,Detroit Pistons,Brooklyn Nets,Brooklyn Nets,23,21.0,29.0,50.0,23.0,33.0,106.0,-0.205098,-0.082039,0.041020,0.162912,0.324575,0.203328,0.008610,0.200176,0.177188,0.262178,0.467905,0.451663,1.069555,0.449298,0.259424,0.523491,0.495593,0.386788,21.0,29.0,50.0,23.0,33.0,106.0
38102,2021-05-14,Utah Jazz,Oklahoma City Thunder,Oklahoma City Thunder,19,32.0,33.0,65.0,18.0,23.0,106.0,0.134516,-0.241151,-0.047448,0.055191,-0.083575,-0.537718,0.134775,-0.231246,-0.397233,-1.065659,-0.780161,-0.458977,0.095559,0.157543,0.473661,-0.426507,-0.511809,-0.310741,32.0,33.0,65.0,18.0,23.0,106.0
21288,2014-11-01,Toronto Raptors,Orlando Magic,Toronto Raptors,38,24.0,36.0,60.0,26.0,23.0,109.0,-0.346560,0.238324,0.070325,0.265163,0.069525,-0.040521,-0.053238,0.214558,0.020190,-0.698422,-0.614834,-0.811864,-0.003596,-0.275755,0.021146,-0.474072,-0.378564,-0.468103,24.0,36.0,60.0,26.0,23.0,109.0
5746,2008-12-23,Denver Nuggets,Portland Trail Blazers,Portland Trail Blazers,20,27.0,20.0,47.0,28.0,14.0,89.0,0.770957,0.828993,0.202203,0.140349,-0.193453,-0.153397,0.699556,0.517692,0.053939,1.294320,0.795665,0.576743,-0.499040,-0.817599,-0.390958,0.751524,0.499366,0.189900,27.0,20.0,47.0,28.0,14.0,89.0
7520,2009-03-15,New Jersey Nets,Los Angeles Clippers,Los Angeles Clippers,23,17.0,29.0,46.0,25.0,23.0,94.0,0.103843,0.091134,0.072070,0.866495,0.251986,0.040749,0.646522,0.228881,0.075462,0.003498,-0.111236,0.043655,-0.901237,-0.737531,-0.263954,-0.077122,-0.285605,-0.118819,17.0,29.0,46.0,25.0,23.0,94.0


### Last 5 Performances

In [42]:
stats = ['q1', 'q2', 'h1', 'q3', 'q4', 'final']
dates = ['_1', '_2', '_3', '_4', '_5']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_5_games = last_15_games[cols].copy()

for stat in stats:
    last_5_games[stat] = 0
    for date in dates:
        last_5_games[stat] = last_5_games[stat] + last_5_games[stat + date]
    
    last_5_games[stat] = last_5_games[stat] / len(dates)

last_5_games = last_5_games.dropna(axis=0)
last_5_games.head()

,date,visitor,home,team,target,q1_1,q1_2,q1_3,q1_4,q1_5,q2_1,q2_2,q2_3,q2_4,q2_5,h1_1,h1_2,h1_3,h1_4,h1_5,q3_1,q3_2,q3_3,q3_4,q3_5,q4_1,q4_2,q4_3,q4_4,q4_5,final_1,final_2,final_3,final_4,final_5,q1_trend_3,q1_trend_5,q1_trend_10,q2_trend_3,q2_trend_5,q2_trend_10,h1_trend_3,h1_trend_5,h1_trend_10,q3_trend_3,q3_trend_5,q3_trend_10,q4_trend_3,q4_trend_5,q4_trend_10,final_trend_3,final_trend_5,final_trend_10,q1,q2,h1,q3,q4,final
15,2006-12-02,Miami Heat,Memphis Grizzlies,Miami Heat,19,25.0,25.0,23.0,23.0,18.0,20.0,26.0,28.0,24.0,20.0,45.0,51.0,51.0,47.0,38.0,19.0,23.0,21.0,31.0,24.0,21.0,29.0,30.0,26.0,24.0,85.0,103.0,102.0,104.0,86.0,0.167292,-0.116746,-0.320512,0.138024,-0.039472,0.126930,0.221792,-0.111045,-0.123846,-0.297706,0.149053,-0.160242,0.552470,0.434865,-0.411891,0.203901,0.145254,-0.277006,22.8,23.6,46.4,23.6,26.0,96.0
16,2006-12-05,Miami Heat,Los Angeles Clippers,Miami Heat,21,25.0,25.0,25.0,23.0,23.0,32.0,20.0,26.0,28.0,24.0,57.0,45.0,51.0,51.0,47.0,19.0,19.0,23.0,21.0,31.0,22.0,21.0,29.0,30.0,26.0,98.0,85.0,103.0,102.0,104.0,0.290787,0.142593,-0.320512,0.359894,0.359894,0.376534,0.477820,0.375409,0.068175,-0.412260,-0.022778,-0.074327,0.082050,0.364302,-0.306047,0.086607,0.356384,-0.048282,24.2,26.0,50.2,22.6,25.6,98.4
17,2006-12-07,Miami Heat,Sacramento Kings,Miami Heat,26,25.0,25.0,25.0,25.0,23.0,19.0,32.0,20.0,26.0,28.0,44.0,57.0,45.0,51.0,51.0,21.0,19.0,19.0,23.0,21.0,32.0,22.0,21.0,29.0,30.0,97.0,98.0,85.0,103.0,102.0,0.290787,0.216690,-0.172318,-0.028379,0.193491,0.110290,0.179120,0.298600,-0.034236,-0.526813,-0.366438,-0.246157,0.258458,0.575991,-0.129639,-0.089335,0.233225,-0.118658,24.6,25.0,49.6,20.6,26.8,97.0
18,2006-12-08,Miami Heat,Denver Nuggets,Miami Heat,35,22.0,25.0,25.0,25.0,25.0,20.0,19.0,32.0,20.0,26.0,42.0,44.0,57.0,45.0,51.0,26.0,21.0,19.0,19.0,23.0,17.0,32.0,22.0,21.0,29.0,93.0,97.0,98.0,85.0,103.0,0.105545,0.179642,-0.172318,-0.028379,-0.072753,-0.006192,0.051106,0.068175,-0.123846,-0.125876,-0.194608,-0.039961,0.023248,0.117332,-0.059076,0.145254,0.074877,0.030892,24.4,23.4,47.8,21.6,24.2,95.2
19,2006-12-11,Toronto Raptors,Miami Heat,Miami Heat,17,32.0,22.0,25.0,25.0,25.0,24.0,20.0,19.0,32.0,20.0,56.0,42.0,44.0,57.0,45.0,35.0,26.0,21.0,19.0,19.0,16.0,17.0,32.0,22.0,21.0,107.0,93.0,97.0,98.0,85.0,0.537776,0.438980,0.049972,-0.472119,-0.139314,0.027089,0.008435,0.196189,0.055374,0.790553,0.217785,0.286517,-0.329568,-0.341328,-0.059076,0.409166,0.145254,0.154051,25.8,23.0,48.8,24.0,21.6,96.0


### Last 10 Perfomances

In [43]:
stats = ['q1', 'q2', 'h1', 'q3', 'q4', 'final']
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_10_games = last_15_games[cols].copy()

for stat in stats:
    last_10_games[stat] = 0
    for date in dates:
        last_10_games[stat] = last_10_games[stat] + last_10_games[stat + date]
    
    last_10_games[stat] = last_10_games[stat] / len(dates)

last_10_games = last_10_games.dropna(axis=0)
last_10_games.head()

,date,visitor,home,team,target,q1_1,q1_2,q1_3,q1_4,q1_5,q1_6,q1_7,q1_8,q1_9,q1_10,q2_1,q2_2,q2_3,q2_4,q2_5,q2_6,q2_7,q2_8,q2_9,q2_10,h1_1,h1_2,h1_3,h1_4,h1_5,h1_6,h1_7,h1_8,h1_9,h1_10,q3_1,q3_2,q3_3,q3_4,q3_5,q3_6,q3_7,q3_8,q3_9,q3_10,q4_1,q4_2,q4_3,q4_4,q4_5,q4_6,q4_7,q4_8,q4_9,q4_10,final_1,final_2,final_3,final_4,final_5,final_6,final_7,final_8,final_9,final_10,q1_trend_3,q1_trend_5,q1_trend_10,q2_trend_3,q2_trend_5,q2_trend_10,h1_trend_3,h1_trend_5,h1_trend_10,q3_trend_3,q3_trend_5,q3_trend_10,q4_trend_3,q4_trend_5,q4_trend_10,final_trend_3,final_trend_5,final_trend_10,q1,q2,h1,q3,q4,final
15,2006-12-02,Miami Heat,Memphis Grizzlies,Miami Heat,19,25.0,25.0,23.0,23.0,18.0,19.0,20.0,22.0,17.0,25.0,20.0,26.0,28.0,24.0,20.0,27.0,22.0,27.0,35.0,17.0,45.0,51.0,51.0,47.0,38.0,46.0,42.0,49.0,52.0,42.0,19.0,23.0,21.0,31.0,24.0,25.0,16.0,14.0,31.0,14.0,21.0,29.0,30.0,26.0,24.0,15.0,16.0,13.0,22.0,16.0,85.0,103.0,102.0,104.0,86.0,86.0,93.0,76.0,105.0,72.0,0.167292,-0.116746,-0.320512,0.138024,-0.039472,0.126930,0.221792,-0.111045,-0.123846,-0.297706,0.149053,-0.160242,0.552470,0.434865,-0.411891,0.203901,0.145254,-0.277006,21.7,24.6,46.3,21.8,21.2,91.2
16,2006-12-05,Miami Heat,Los Angeles Clippers,Miami Heat,21,25.0,25.0,25.0,23.0,23.0,18.0,19.0,20.0,22.0,17.0,32.0,20.0,26.0,28.0,24.0,20.0,27.0,22.0,27.0,35.0,57.0,45.0,51.0,51.0,47.0,38.0,46.0,42.0,49.0,52.0,19.0,19.0,23.0,21.0,31.0,24.0,25.0,16.0,14.0,31.0,22.0,21.0,29.0,30.0,26.0,24.0,15.0,16.0,13.0,22.0,98.0,85.0,103.0,102.0,104.0,86.0,86.0,93.0,76.0,105.0,0.290787,0.142593,-0.320512,0.359894,0.359894,0.376534,0.477820,0.375409,0.068175,-0.412260,-0.022778,-0.074327,0.082050,0.364302,-0.306047,0.086607,0.356384,-0.048282,21.7,26.1,47.8,22.3,21.8,93.8
17,2006-12-07,Miami Heat,Sacramento Kings,Miami Heat,26,25.0,25.0,25.0,25.0,23.0,23.0,18.0,19.0,20.0,22.0,19.0,32.0,20.0,26.0,28.0,24.0,20.0,27.0,22.0,27.0,44.0,57.0,45.0,51.0,51.0,47.0,38.0,46.0,42.0,49.0,21.0,19.0,19.0,23.0,21.0,31.0,24.0,25.0,16.0,14.0,32.0,22.0,21.0,29.0,30.0,26.0,24.0,15.0,16.0,13.0,97.0,98.0,85.0,103.0,102.0,104.0,86.0,86.0,93.0,76.0,0.290787,0.216690,-0.172318,-0.028379,0.193491,0.110290,0.179120,0.298600,-0.034236,-0.526813,-0.366438,-0.246157,0.258458,0.575991,-0.129639,-0.089335,0.233225,-0.118658,22.5,24.5,47.0,21.3,22.8,93.0
18,2006-12-08,Miami Heat,Denver Nuggets,Miami Heat,35,22.0,25.0,25.0,25.0,25.0,23.0,23.0,18.0,19.0,20.0,20.0,19.0,32.0,20.0,26.0,28.0,24.0,20.0,27.0,22.0,42.0,44.0,57.0,45.0,51.0,51.0,47.0,38.0,46.0,42.0,26.0,21.0,19.0,19.0,23.0,21.0,31.0,24.0,25.0,16.0,17.0,32.0,22.0,21.0,29.0,30.0,26.0,24.0,15.0,16.0,93.0,97.0,98.0,85.0,103.0,102.0,104.0,86.0,86.0,93.0,0.105545,0.179642,-0.172318,-0.028379,-0.072753,-0.006192,0.051106,0.068175,-0.123846,-0.125876,-0.194608,-0.039961,0.023248,0.117332,-0.059076,0.145254,0.074877,0.030892,22.5,23.8,46.3,22.5,23.2,94.7
19,2006-12-11,Toronto Raptors,Miami Heat,Miami Heat,17,32.0,22.0,25.0,25.0,25.0,25.0,23.0,23.0,18.0,19.0,24.0,20.0,19.0,32.0,20.0,26.0,28.0,24.0,20.0,27.0,56.0,42.0,44.0,57.0,45.0,51.0,51.0,47.0,38.0,46.0,35.0,26.0,21.0,19.0,19.0,23.0,21.0,31.0,24.0,25.0,16.0,17.0,32.0,22.0,21.0,29.0,30.0,26.0,24.0,15.0,107.0,93.0,97.0,98.0,85.0,103.0,102.0,104.0,86.0,86.0,0.537776,0.438980,0.049972,-0.472119,-0.139314,0.027089,0.008435,0.196189,0.055374,0.790553,0.217785,0.286517,-0.329568,-0.341328,-0.059076,0.409166,0.145254,0.154051,23.7,24.0,47.7,24.4,23.2,96.1


### Last 15 Performances (Unweighted)

In [44]:
stats = ['q1', 'q2', 'h1', 'q3', 'q4', 'final']
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_15_games_unweighted = last_15_games[cols].copy()

for stat in stats:
    last_15_games_unweighted[stat] = 0
    for date in dates:
        last_15_games_unweighted[stat] = last_15_games_unweighted[stat] + last_15_games_unweighted[stat + date]
    
    last_15_games_unweighted[stat] = last_15_games_unweighted[stat] / len(dates)

last_15_games_unweighted = last_15_games_unweighted.dropna(axis=0)
last_15_games_unweighted.head()

,date,visitor,home,team,target,q1_1,q1_2,q1_3,q1_4,q1_5,q1_6,q1_7,q1_8,q1_9,q1_10,q1_11,q1_12,q1_13,q1_14,q1_15,q2_1,q2_2,q2_3,q2_4,q2_5,q2_6,q2_7,q2_8,q2_9,q2_10,q2_11,q2_12,q2_13,q2_14,q2_15,h1_1,h1_2,h1_3,h1_4,h1_5,h1_6,h1_7,h1_8,h1_9,h1_10,h1_11,h1_12,h1_13,h1_14,h1_15,q3_1,q3_2,q3_3,q3_4,q3_5,q3_6,q3_7,q3_8,q3_9,q3_10,q3_11,q3_12,q3_13,q3_14,q3_15,q4_1,q4_2,q4_3,q4_4,q4_5,q4_6,q4_7,q4_8,q4_9,q4_10,q4_11,q4_12,q4_13,q4_14,q4_15,final_1,final_2,final_3,final_4,final_5,final_6,final_7,final_8,final_9,final_10,final_11,final_12,final_13,final_14,final_15,q1_trend_3,q1_trend_5,q1_trend_10,q2_trend_3,q2_trend_5,q2_trend_10,h1_trend_3,h1_trend_5,h1_trend_10,q3_trend_3,q3_trend_5,q3_trend_10,q4_trend_3,q4_trend_5,q4_trend_10,final_trend_3,final_trend_5,final_trend_10,q1,q2,h1,q3,q4,final
15,2006-12-02,Miami Heat,Memphis Grizzlies,Miami Heat,19,25.0,25.0,23.0,23.0,18.0,19.0,20.0,22.0,17.0,25.0,22.0,11.0,29.0,21.0,16.0,20.0,26.0,28.0,24.0,20.0,27.0,22.0,27.0,35.0,17.0,21.0,29.0,28.0,22.0,14.0,45.0,51.0,51.0,47.0,38.0,46.0,42.0,49.0,52.0,42.0,43.0,40.0,57.0,43.0,30.0,19.0,23.0,21.0,31.0,24.0,25.0,16.0,14.0,31.0,14.0,33.0,25.0,16.0,22.0,21.0,21.0,29.0,30.0,26.0,24.0,15.0,16.0,13.0,22.0,16.0,37.0,25.0,25.0,26.0,15.0,85.0,103.0,102.0,104.0,86.0,86.0,93.0,76.0,105.0,72.0,113.0,90.0,98.0,91.0,66.0,0.167292,-0.116746,-0.320512,0.138024,-0.039472,0.126930,0.221792,-0.111045,-0.123846,-0.297706,0.149053,-0.160242,0.552470,0.434865,-0.411891,0.203901,0.145254,-0.277006,21.066667,24.000000,45.066667,22.333333,22.666667,91.333333
16,2006-12-05,Miami Heat,Los Angeles Clippers,Miami Heat,21,25.0,25.0,25.0,23.0,23.0,18.0,19.0,20.0,22.0,17.0,25.0,22.0,11.0,29.0,21.0,32.0,20.0,26.0,28.0,24.0,20.0,27.0,22.0,27.0,35.0,17.0,21.0,29.0,28.0,22.0,57.0,45.0,51.0,51.0,47.0,38.0,46.0,42.0,49.0,52.0,42.0,43.0,40.0,57.0,43.0,19.0,19.0,23.0,21.0,31.0,24.0,25.0,16.0,14.0,31.0,14.0,33.0,25.0,16.0,22.0,22.0,21.0,29.0,30.0,26.0,24.0,15.0,16.0,13.0,22.0,16.0,37.0,25.0,25.0,26.0,98.0,85.0,103.0,102.0,104.0,86.0,86.0,93.0,76.0,105.0,72.0,113.0,90.0,98.0,91.0,0.290787,0.142593,-0.320512,0.359894,0.359894,0.376534,0.477820,0.375409,0.068175,-0.412260,-0.022778,-0.074327,0.082050,0.364302,-0.306047,0.086607,0.356384,-0.048282,21.666667,25.200000,46.866667,22.200000,23.133333,93.466667
17,2006-12-07,Miami Heat,Sacramento Kings,Miami Heat,26,25.0,25.0,25.0,25.0,23.0,23.0,18.0,19.0,20.0,22.0,17.0,25.0,22.0,11.0,29.0,19.0,32.0,20.0,26.0,28.0,24.0,20.0,27.0,22.0,27.0,35.0,17.0,21.0,29.0,28.0,44.0,57.0,45.0,51.0,51.0,47.0,38.0,46.0,42.0,49.0,52.0,42.0,43.0,40.0,57.0,21.0,19.0,19.0,23.0,21.0,31.0,24.0,25.0,16.0,14.0,31.0,14.0,33.0,25.0,16.0,32.0,22.0,21.0,29.0,30.0,26.0,24.0,15.0,16.0,13.0,22.0,16.0,37.0,25.0,25.0,97.0,98.0,85.0,103.0,102.0,104.0,86.0,86.0,93.0,76.0,105.0,72.0,113.0,90.0,98.0,0.290787,0.216690,-0.172318,-0.028379,0.193491,0.110290,0.179120,0.298600,-0.034236,-0.526813,-0.366438,-0.246157,0.258458,0.575991,-0.129639,-0.089335,0.233225,-0.118658,21.933333,25.000000,46.933333,22.133333,23.533333,93.866667
18,2006-12-08,Miami Heat,Denver Nuggets,Miami Heat,35,22.0,25.0,25.0,25.0,25.0,23.0,23.0,18.0,19.0,20.0,22.0,17.0,25.0,22.0,11.0,20.0,19.0,32.0,20.0,26.0,28.0,24.0,20.0,27.0,22.0,27.0,35.0,17.0,21.0,29.0,42.0,44.0,57.0,45.0,51.0,51.0,47.0,38.0,46.0,42.0,49.0,52.0,42.0,43.0,40.0,26.0,21.0,19.0,19.0,23.0,21.0,31.0,24.0,25.0,16.0,14.0,31.0,14.0,33.0,25.0,17.0,32.0,22.0,21.0,29.0,30.0,26.0,24.0,15.0,16.0,13.0,22.0,16.0,37.0,25.0,93.0,97.0,98.0,85.0,103.0,102.0,104.0,86.0,86.0,93.0,76.0,105.0,72.0,113.0,90.0,0.105545,0.179642,-0.172318,-0.028379,-0.072753,-0.006192,0.051106,0.068175,-0.123846,-0.125876,-0.194608,-0.039961,0.023248,0.117332,-0.059076,0.145254,0.074877,0.030892,21.466667,24.466667,45.933333,22.800000,23.000000,93.533333
19,2006-12-11,Toronto Raptors,Miami Heat,Miami Heat,17,32.0,22.0,25.0,25.0,25.0,25.0,23.0,23.0,18.0,19.0,20.0,22.0,17.0,25.0,22.0,24.0,20.0,19.0,32.0,20.0,26.0,28.0,24.0,20.0,27.0,22.0,27.0,35.0,17.0,21.0,56.0,42.0,44.0,57.0,45.0,51.0,5

### Last 15 Performances (Weighted)

In [45]:
stats = ['q1', 'q2', 'h1', 'q3', 'q4', 'final']
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_15_games_weighted = last_15_games[cols].copy()

for stat in stats:
    last_15_games_weighted[stat] = 0
    for date in dates:
        weight = ((int(date.strip('_')) - 1) // 5) + 1
        last_15_games_weighted[stat] = (last_15_games_weighted[stat] + last_15_games_weighted[stat + date]) * weight
    
    last_15_games_weighted[stat] = last_15_games_weighted[stat] / len(dates)

last_15_games_weighted = last_15_games_weighted.dropna(axis=0)
last_15_games_weighted.head()

,date,visitor,home,team,target,q1_1,q1_2,q1_3,q1_4,q1_5,q1_6,q1_7,q1_8,q1_9,q1_10,q1_11,q1_12,q1_13,q1_14,q1_15,q2_1,q2_2,q2_3,q2_4,q2_5,q2_6,q2_7,q2_8,q2_9,q2_10,q2_11,q2_12,q2_13,q2_14,q2_15,h1_1,h1_2,h1_3,h1_4,h1_5,h1_6,h1_7,h1_8,h1_9,h1_10,h1_11,h1_12,h1_13,h1_14,h1_15,q3_1,q3_2,q3_3,q3_4,q3_5,q3_6,q3_7,q3_8,q3_9,q3_10,q3_11,q3_12,q3_13,q3_14,q3_15,q4_1,q4_2,q4_3,q4_4,q4_5,q4_6,q4_7,q4_8,q4_9,q4_10,q4_11,q4_12,q4_13,q4_14,q4_15,final_1,final_2,final_3,final_4,final_5,final_6,final_7,final_8,final_9,final_10,final_11,final_12,final_13,final_14,final_15,q1_trend_3,q1_trend_5,q1_trend_10,q2_trend_3,q2_trend_5,q2_trend_10,h1_trend_3,h1_trend_5,h1_trend_10,q3_trend_3,q3_trend_5,q3_trend_10,q4_trend_3,q4_trend_5,q4_trend_10,final_trend_3,final_trend_5,final_trend_10,q1,q2,h1,q3,q4,final
15,2006-12-02,Miami Heat,Memphis Grizzlies,Miami Heat,19,25.0,25.0,23.0,23.0,18.0,19.0,20.0,22.0,17.0,25.0,22.0,11.0,29.0,21.0,16.0,20.0,26.0,28.0,24.0,20.0,27.0,22.0,27.0,35.0,17.0,21.0,29.0,28.0,22.0,14.0,45.0,51.0,51.0,47.0,38.0,46.0,42.0,49.0,52.0,42.0,43.0,40.0,57.0,43.0,30.0,19.0,23.0,21.0,31.0,24.0,25.0,16.0,14.0,31.0,14.0,33.0,25.0,16.0,22.0,21.0,21.0,29.0,30.0,26.0,24.0,15.0,16.0,13.0,22.0,16.0,37.0,25.0,25.0,26.0,15.0,85.0,103.0,102.0,104.0,86.0,86.0,93.0,76.0,105.0,72.0,113.0,90.0,98.0,91.0,66.0,0.167292,-0.116746,-0.320512,0.138024,-0.039472,0.126930,0.221792,-0.111045,-0.123846,-0.297706,0.149053,-0.160242,0.552470,0.434865,-0.411891,0.203901,0.145254,-0.277006,79377.8,87751.6,167129.4,83271.0,83742.0,339067.2
16,2006-12-05,Miami Heat,Los Angeles Clippers,Miami Heat,21,25.0,25.0,25.0,23.0,23.0,18.0,19.0,20.0,22.0,17.0,25.0,22.0,11.0,29.0,21.0,32.0,20.0,26.0,28.0,24.0,20.0,27.0,22.0,27.0,35.0,17.0,21.0,29.0,28.0,22.0,57.0,45.0,51.0,51.0,47.0,38.0,46.0,42.0,49.0,52.0,42.0,43.0,40.0,57.0,43.0,19.0,19.0,23.0,21.0,31.0,24.0,25.0,16.0,14.0,31.0,14.0,33.0,25.0,16.0,22.0,22.0,21.0,29.0,30.0,26.0,24.0,15.0,16.0,13.0,22.0,16.0,37.0,25.0,25.0,26.0,98.0,85.0,103.0,102.0,104.0,86.0,86.0,93.0,76.0,105.0,72.0,113.0,90.0,98.0,91.0,0.290787,0.142593,-0.320512,0.359894,0.359894,0.376534,0.477820,0.375409,0.068175,-0.412260,-0.022778,-0.074327,0.082050,0.364302,-0.306047,0.086607,0.356384,-0.048282,82116.0,90955.4,173071.4,81950.0,86837.8,344321.6
17,2006-12-07,Miami Heat,Sacramento Kings,Miami Heat,26,25.0,25.0,25.0,25.0,23.0,23.0,18.0,19.0,20.0,22.0,17.0,25.0,22.0,11.0,29.0,19.0,32.0,20.0,26.0,28.0,24.0,20.0,27.0,22.0,27.0,35.0,17.0,21.0,29.0,28.0,44.0,57.0,45.0,51.0,51.0,47.0,38.0,46.0,42.0,49.0,52.0,42.0,43.0,40.0,57.0,21.0,19.0,19.0,23.0,21.0,31.0,24.0,25.0,16.0,14.0,31.0,14.0,33.0,25.0,16.0,32.0,22.0,21.0,29.0,30.0,26.0,24.0,15.0,16.0,13.0,22.0,16.0,37.0,25.0,25.0,97.0,98.0,85.0,103.0,102.0,104.0,86.0,86.0,93.0,76.0,105.0,72.0,113.0,90.0,98.0,0.290787,0.216690,-0.172318,-0.028379,0.193491,0.110290,0.179120,0.298600,-0.034236,-0.526813,-0.366438,-0.246157,0.258458,0.575991,-0.129639,-0.089335,0.233225,-0.118658,85285.6,88944.8,174230.4,81072.2,93096.2,349630.0
18,2006-12-08,Miami Heat,Denver Nuggets,Miami Heat,35,22.0,25.0,25.0,25.0,25.0,23.0,23.0,18.0,19.0,20.0,22.0,17.0,25.0,22.0,11.0,20.0,19.0,32.0,20.0,26.0,28.0,24.0,20.0,27.0,22.0,27.0,35.0,17.0,21.0,29.0,42.0,44.0,57.0,45.0,51.0,51.0,47.0,38.0,46.0,42.0,49.0,52.0,42.0,43.0,40.0,26.0,21.0,19.0,19.0,23.0,21.0,31.0,24.0,25.0,16.0,14.0,31.0,14.0,33.0,25.0,17.0,32.0,22.0,21.0,29.0,30.0,26.0,24.0,15.0,16.0,13.0,22.0,16.0,37.0,25.0,93.0,97.0,98.0,85.0,103.0,102.0,104.0,86.0,86.0,93.0,76.0,105.0,72.0,113.0,90.0,0.105545,0.179642,-0.172318,-0.028379,-0.072753,-0.006192,0.051106,0.068175,-0.123846,-0.125876,-0.194608,-0.039961,0.023248,0.117332,-0.059076,0.145254,0.074877,0.030892,85850.2,87118.6,172968.8,80601.8,90003.8,348337.2
19,2006-12-11,Toronto Raptors,Miami Heat,Miami Heat,17,32.0,22.0,25.0,25.0,25.0,25.0,23.0,23.0,18.0,19.0,20.0,22.0,17.0,25.0,22.0,24.0,20.0,19.0,32.0,20.0,26.0,28.0,24.0,20.0,27.0,22.0,27.0,35.0,17.0,21.0,56.0,42.0,44.0,57.0,45.0,51.0,51.0,47.0,38.0,46.0,42.0,49.0,52.0,42.0,4

## Correlations of performances (last 1, last 5, last 10)

In [49]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3Q pt (unweighted)
for col in last_15_games_weighted.iloc[:, 4:]:
    corr_p = pearsonr(last_15_games_weighted['target'], last_15_games_weighted[col])
    row = {'last': '15_weighted', 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)

# Correlations for last 15 game stats vs 3Q pt (unweighted)
for col in last_15_games_unweighted.iloc[:, 4:]:
    corr_p = pearsonr(last_15_games_unweighted['target'], last_15_games_unweighted[col])
    row = {'last': '15_unweighted', 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)

# Correlations for last 10 game stats vs 3Q pt 
for col in last_10_games.iloc[:, 4:]:
    corr_p = pearsonr(last_10_games['target'], last_10_games[col])
    row = {'last': 10, 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Correlations for last 10 game stats vs 3Q pt
for col in last_5_games.iloc[:, 4:]:
    corr_p = pearsonr(last_5_games['target'], last_5_games[col])
    row = {'last': 5, 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Correlations for last game stats vs 3Q pt 
for col in last_game.iloc[:, 4:]:
    corr_p = pearsonr(last_game['target'], last_game[col])
    row = {'last': 1, 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)

# Correlations for current game stats vs 3Q pt
for col in df.iloc[:, 4:11]:
    corr_p = pearsonr(df['target'], df[col])
    row = {'last': 'current', 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print each correlation
for stat in stats + [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))] + ['target']:
    print(f'Stat: {stat}')
    print(corr_df[(corr_df['stat'] == stat) & (corr_df['p-value'] < .05)].set_index(['last']).drop(['stat'], axis=1))
    print('\n')

Stat: q1
               corr  p-value
last                        
15_weighted    0.19      0.0
15_unweighted  0.21      0.0
10             0.20      0.0
5              0.17      0.0
1              0.09      0.0
current        0.10      0.0


Stat: q2
               corr  p-value
last                        
15_weighted    0.19      0.0
15_unweighted  0.22      0.0
10             0.20      0.0
5              0.17      0.0
1              0.10      0.0
current        0.09      0.0


Stat: h1
               corr  p-value
last                        
15_weighted    0.22      0.0
15_unweighted  0.24      0.0
10             0.23      0.0
5              0.21      0.0
1              0.13      0.0
current        0.13      0.0


Stat: q3
               corr  p-value
last                        
15_weighted    0.21      0.0
15_unweighted  0.24      0.0
10             0.22      0.0
5              0.19      0.0
1              0.10      0.0
current        1.00      0.0


Stat: q4
               corr